In [1]:
import datetime as dt
import pandas as pd

from processor.dart import DART_FUNDAMENTAL_PROCESSOR,DART_INFO_PROCESSOR
from processor.fdr import FDR_INFO_PROCESSOR, FDR_OHLCV_PREPROCESSOR
from processor.krx import KRX_INFO_PROCESSOR

from loader.pykrx_loader import get_pykrx_loader

from processor.factor.fundamental import FUNDAMENTAL_FACTOR_PROCESSOR
from processor.factor.technical import MOVING_AVERAGE_FACTOR_PROCESSOR
from processor.factor.trader import TRADER_FACTOR_PROCESSOR

from processor.factor.filter import FACTOR_FILTER

In [2]:
CFG = {
    "FUNDAMENTAL": {
        "FACTOR_PROCESSOR_CFG": {
            "ohlcv_recent_n": 10,
        },
        "FACTOR_ANALYSIS_CFG": {
            "profit_recent_n": 20,
        },
        "FACTOR_FILTER_CFG": {
            "quantile_conditions": [
                ("CLR", 0.0, 1.0),
                ("TLR", 0.1, 0.7),
                ("NPP", 0.3, 0.8),
                ("OPP", 0.3, 0.7),
                ("TAPP", 0.0, 0.5),
                ("TEPP", 0.0, 0.5),
                ("CAPP", 0.0, 0.4),
                ("CEPP", 0.1, 0.5),
            ]
        },
    },
    "MOVING_AVERAGE": {
        "FACTOR_PROCESSOR_CFG": {
            "factors": ["Close", "VolumeRotation"],
            "short_term_window": round((7 * 2) * (5 / 7)),
            "long_term_window": round((7 * 8) * (5 / 7)),
            "signal_recent_n": 20,
        },
        "FACTOR_ANALYSIS_CFG": {
            "profit_recent_n": 20,
        },
        "FACTOR_FILTER_CFG": {
            "quantile_conditions": [
                ("Close", 0.6, 0.9),
                ("VolumeRotation", 0.0, 0.3),
            ]
        },
    },
    "TRADER": {
        "FACTOR_PROCESSOR_CFG": {
            "start": (dt.date.today() - dt.timedelta(days=7)).strftime("%Y-%m-%d"),
            "end": (dt.date.today() - dt.timedelta(days=1)).strftime("%Y-%m-%d"),
            # "end": dt.date.today().strftime("%Y-%m-%d"),
        },
        "FACTOR_FILTER_CFG": {
            "absolute_conditions": [
                "Corp > 0",
                "Foreign > 0",
                "Indivisual < 0",
            ]
        },
    },
}

In [3]:
# dart_fundamental
dart_fundamental_df = pd.read_csv("./data/dart_fundamental_df.csv", index_col=0)
dart_fundamental_processor = DART_FUNDAMENTAL_PROCESSOR(dart_fundamental_df)

In [4]:
# dart_info
dart_info_df = pd.read_csv("./data/dart_info_df.csv", index_col=0)
dart_info_processor = DART_INFO_PROCESSOR(dart_info_df)

In [5]:
# fdr_info
fdr_info_df = pd.read_csv("./data/fdr_info_df.csv", index_col=0)
fdr_info_processor = FDR_INFO_PROCESSOR(fdr_info_df)

In [6]:
# krx_info
krx_info_df = pd.read_csv("./data/krx_info_df.csv", index_col=0)
krx_info_processor = KRX_INFO_PROCESSOR(krx_info_df)

In [7]:
# fdr_ohlcv
fdr_ohlcv_df = pd.read_csv("./data/fdr_ohlcv_df.csv", index_col=0)
fdr_ohlcv_preprocessor = FDR_OHLCV_PREPROCESSOR(fdr_ohlcv_df)
ohlcv_df = fdr_ohlcv_preprocessor.get_pps_fdr_ohlcv_df()

In [8]:
# Fundamental
fundamental_factor_processor = FUNDAMENTAL_FACTOR_PROCESSOR(ohlcv_df, dart_fundamental_processor, fdr_info_processor)
fundamental_factor_df = fundamental_factor_processor.get_fundamental_factor_df(
    CFG["FUNDAMENTAL"]["FACTOR_PROCESSOR_CFG"]
)

factor_filter = FACTOR_FILTER(fundamental_factor_df)
fundamental_stockcodes = factor_filter.get_filtered_stockcodes(CFG["FUNDAMENTAL"]["FACTOR_FILTER_CFG"])

In [9]:
# Technical
## Moving Average
moving_average_factor_processor = MOVING_AVERAGE_FACTOR_PROCESSOR(ohlcv_df, fdr_info_processor)
moving_average_factors_df = moving_average_factor_processor.get_moving_average_factor_df(
    CFG["MOVING_AVERAGE"]["FACTOR_PROCESSOR_CFG"]
)

factor_filter = FACTOR_FILTER(moving_average_factors_df)
technical_stockcodes = factor_filter.get_filtered_stockcodes(
    CFG["MOVING_AVERAGE"]["FACTOR_FILTER_CFG"], fundamental_stockcodes
)

In [10]:
# Trader
pykrx_loader = get_pykrx_loader()

trader_factor_processor = TRADER_FACTOR_PROCESSOR(pykrx_loader)
trader_factor_df = trader_factor_processor.get_trader_factor_df(
    technical_stockcodes, CFG["TRADER"]["FACTOR_PROCESSOR_CFG"]
)
factor_filter = FACTOR_FILTER(trader_factor_df)
final_stockcodes = factor_filter.get_filtered_stockcodes(CFG["TRADER"]["FACTOR_FILTER_CFG"], technical_stockcodes)

In [11]:
final_stockcodes

set()